In [5]:
import spacy
from nltk.corpus import wordnet as wn
import nltk
nltk.download('wordnet')

/home/dell/.virtualenvs/event_part/lib/python3.9/site-packages/torch/cuda/__init__.py:52: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at  /pytorch/c10/cuda/CUDAFunctions.cpp:100.)
  return torch._C._cuda_getDeviceCount() > 0
[nltk_data] Downloading package wordnet to /home/dell/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [6]:
nltk.download('omw-1.4')

[nltk_data] Downloading package omw-1.4 to /home/dell/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [7]:
np = spacy.load('en_core_web_sm')

In [9]:
parsed = np("He eats orange and apple.")
type(parsed)

spacy.tokens.doc.Doc

In [10]:
spacy.displacy.render(parsed, jupyter=True)

In [7]:
parsed = np("He is boy")
spacy.displacy.render(parsed, jupyter=True)

In [12]:
for each in parsed:
    #print(each)
    print(f"{each.head}------{each.dep_}------>{each.text}({each.pos_})")
    print("----------------")
pairs = [(token.head,token.dep_,token.text) for token in parsed]
for pair in pairs:
    print(pair)

eats------nsubj------>He(PRON)
----------------
eats------ROOT------>eats(VERB)
----------------
eats------dobj------>orange(NOUN)
----------------
orange------cc------>and(CCONJ)
----------------
orange------conj------>apple(NOUN)
----------------
eats------punct------>.(PUNCT)
----------------
(eats, 'nsubj', 'He')
(eats, 'ROOT', 'eats')
(eats, 'dobj', 'orange')
(orange, 'cc', 'and')
(orange, 'conj', 'apple')
(eats, 'punct', '.')


In [10]:
dep_a, dep_b = wn.synsets("he")[0], wn.synsets("is")[0]
print(dep_a,dep_b)

Synset('helium.n.01') Synset('be.v.01')


In [11]:
dep_a.path_similarity(dep_b)

0.125

In [9]:
sparta = wn.synsets("cat")[0]
be = wn.synsets("he")[0]
print(sparta.path_similarity(be))

0.058823529411764705


In [2]:
def similarity_word(pair_A, pair_B):
    
    #getting head and dependent texts 
    head_a, head_b = pair_A[0].text, pair_B[0].text
    dep_a, dep_b = pair_A[2].text, pair_B[2].text
    
    head = 1 if head_a == head_b else 0
    
#     if head_a == head_b:
#         head = 1
#     else:
#         try:
#             #WordNet synsets for heads
#             head_a, head_b = wn.synsets(head_a)[0], wn.synsets(head_b)[0]

#             #path based similarity (Li et. al) for head
#             head = head_a.path_similarity(head_b)
            
#             head = 0 if head is None else head  
        
#         except Exception:
#             head = 0
  
    dep = 1 if dep_a == dep_b else 0
    
#     if dep_a == dep_b:
#         dep = 1
#     else:
#         try:
#             #WordNet synsets for dependent
#             dep_a, dep_b = wn.synsets(dep_a)[0], wn.synsets(dep_b)[0]

#             #path based similarity (Li et. al) for dependent
#             dep = dep_a.path_similarity(dep_b)
            
#             dep = 0 if dep is None else dep

#         except Exception:
#             dep = 0
    
#     print(f"{dep_a}:{dep_b}--{head} {dep}")        
    
    return head + dep
    
#check with diff values
similarity_tag = lambda a, b : 1 if a == b else 0

def semantic_similarity(document_1, document_2):
    
    #parsing documets using spaCy English language parser
    tokens_1,tokens_2 = np(document_1), np(document_2)
    
    #seperating dependency pairs and tags from tokens
    pairs_1 = [(token.head,token.dep_,token) for token in tokens_1]
    pairs_2 = [(token.head,token.dep_,token) for token in tokens_2]
    
    score = 0
    
    #calculating score 
    for pair_A in pairs_1:
        
        for pair_B in pairs_2:
            
            score += similarity_word(pair_A, pair_B) * similarity_tag(pair_A[1], pair_B[1])
    
    #averaging score 
    score = score / (len(tokens_1) + len(tokens_2))
    
    return score 

In [12]:
semantic_similarity("I am a flower.","I am flower.")


0.8888888888888888

In [18]:
spacy.displacy.render(np(""),jupyter=True)

In [13]:
spacy.displacy.render(np("I haven't watched television for ages."),jupyter=True)

In [20]:
def review_to_wordlist(review, remove_stopwords=False):
    # Clean the text, with the option to remove stopwords.
    
    # Convert words to lower case and split them
    words = review.lower().split()

    # Optionally remove stop words (true by default)
    if remove_stopwords:
        stops = set(stopwords.words("english"))
        words = [w for w in words if not w in stops]
    
    review_text = " ".join(words)

    # Clean the text
#     review_text = re.sub(r"@[A-Za-z0-9_]*", "", review_text)
    review_text = re.sub(r"[^A-Za-z0-9(),!.?\'\`]", " ", review_text)
#     review_text = re.sub(r"\'s", "'s", review_text)
    review_text = re.sub(r"\'ve", "'ve", review_text)
    review_text = re.sub(r"n\'t", "'t", review_text)
    review_text = re.sub(r"\'re", "'re", review_text)
    review_text = re.sub(r"\'d", "'d", review_text)
    review_text = re.sub(r"\'ll", "'ll", review_text)
    review_text = re.sub(r",", " ", review_text)
    review_text = re.sub(r"\.", " ", review_text)
    review_text = re.sub(r"!", " ", review_text)
    review_text = re.sub(r"\(", " ", review_text)
    review_text = re.sub(r"\)", " ", review_text)
    review_text = re.sub(r"\?", " ", review_text)
    review_text = re.sub(r"\s{2,}", " ", review_text)
    
    words = review_text.split()
    
    #Shorten words to their stems
    stemmer = SnowballStemmer('english')
    stemmed_words = [stemmer.stem(word) for word in words]
    
    review_text = " ".join(stemmed_words)
    
    # Return a list of words
    return review_text

In [21]:
from collections import Counter
import pandas as pd
import re

data_1 = pd.read_csv("data/STS_Data1213.csv")
data_2 = pd.read_csv("data/STS_Data1517.csv")
data_1 = data_1.iloc[:,2:]
data_2 = data_2.iloc[:,1:]
data = pd.concat([data_1,data_2],ignore_index=True)
data.head()
print(len(data_1))
print(len(data_2))
print(len(data))

4608
320
4928


In [22]:
tags = Counter()
for index, each in data.iterrows():
    
    sent1 = re.sub(r"[\s]+", " ", each[0])
    sent2 = re.sub(r"[\s]+", " ", each[1])
    
    sent1 = np(sent1)
    sent2 = np(sent2)
    
    for toks in sent1:
#         if toks.dep_ == '':
#             print(toks.sent)
#             print(f"{toks.head}------{toks.dep_}------>{toks.text}({toks.pos_})")
        tags[toks.dep_] += 1
        
    for toks in sent2:
#         if toks.dep_ == '':
#             print(toks.sent)
#             print(f"{toks.head}------{toks.dep_}------>{toks.text}({toks.pos_})")
        tags[toks.dep_] += 1

In [23]:
len(tags)

44

In [24]:
keys = {value:key for key,value in enumerate(sorted(tags.keys()))}

In [25]:
keys

{'ROOT': 0,
 'acl': 1,
 'acomp': 2,
 'advcl': 3,
 'advmod': 4,
 'agent': 5,
 'amod': 6,
 'appos': 7,
 'attr': 8,
 'aux': 9,
 'auxpass': 10,
 'case': 11,
 'cc': 12,
 'ccomp': 13,
 'compound': 14,
 'conj': 15,
 'csubj': 16,
 'csubjpass': 17,
 'dative': 18,
 'dep': 19,
 'det': 20,
 'dobj': 21,
 'expl': 22,
 'intj': 23,
 'mark': 24,
 'neg': 25,
 'nmod': 26,
 'npadvmod': 27,
 'nsubj': 28,
 'nsubjpass': 29,
 'nummod': 30,
 'oprd': 31,
 'parataxis': 32,
 'pcomp': 33,
 'pobj': 34,
 'poss': 35,
 'preconj': 36,
 'predet': 37,
 'prep': 38,
 'prt': 39,
 'punct': 40,
 'quantmod': 41,
 'relcl': 42,
 'xcomp': 43}

In [26]:
import json

with open("data/tags.json","w") as fl:
    fl.write(json.dumps(keys,indent=4))

In [27]:
data.score = data.score/5
print(len(data))
for i in range(len(data)):
    data.iloc[i,0] = re.sub(r"[\s]+", " ", data.iloc[i,0])
    data.iloc[i,1] = re.sub(r"[\s]+", " ", data.iloc[i,1])

4928


In [28]:
data.head()
print(len(data))

4928


In [29]:
data.to_csv("data/STS_Final.csv")